# Calculating a Typical Meteorological Year
<br>This notebook walks through the process of calculating a [Typical Meteorological Year](https://nsrdb.nrel.gov/data-sets/tmy), an hourly dataset used for applications in energy and building systems modeling. Because this represents average rather than extreme conditions, an TMY dataset is not suited for designing systems to meet the worst-case conditions occurring at a location. 

The TMY methodology here mirrors that of the Sandia/NREL TMY3 methodology, and uses historic and projected downscaled climate data available through the Cal-Adapt: Analytics Engine catalog. As this methodology heavily weights the solar radiation input data, be aware that the final selection of "typical" months may not be typical for other variables. 

**Intended Application** As a user, I want to <span style="color:#FF0000">**generate a typical meteorological year file**</span> for a location of interest:
- Understand the methods that are involved in generating a TMY dataset
- Visualize the TMY dataset across all input variables
- Export the TMY dataset for available models for input into my workflow

**Note**: 
1. This notebook is a **full demonstration** of the Typical Meteorological Year methodology, for full transparency. For practical generation of a TMY dataset, a user <span style="color:#FF0000">**only needs to provide 2 elements**</span>: the **location**, and **reference time period**. These selections are highlighted below for you. 
2. The Analytics Engine at present has an *Average Meteorological Year* functionality. The methods shown throughout this notebook will soon replace the underlying backend `climakitae` code for the AMY in order to better address our user needs, i.e., we are working to replace the AMY with the TMY methods. We provide this walkthrough to demonstrate confidence in the "AMY to TMY" conversion process for our users in the meantime. 

**Runtime**: With the default settings, this notebook takes approximately **30 minutes** to run from start to finish. Modifications to selections may increase the runtime.

### Step 0: Set-up

Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
#import climakitae as ck
from tmy_functions import TMY
#from climakitae.util.utils import (
#    convert_to_local_time,
#    get_closest_gridcell,
#)
#from climakitae.core.data_export import write_tmy_file
#from climakitae.core.data_interface import get_data
#import climakitaegui as ckg

#import pandas as pd
#import xarray as xr
#import numpy as np
#import pkg_resources
#from tqdm.auto import tqdm  # Progress bar

import warnings

warnings.filterwarnings("ignore")

### Step 1: Grab and process all required input data

The [TMY3 method](https://www.nrel.gov/docs/fy08osti/43156.pdf) selects a "typical" month based on ten daily variables: max, min, and mean air and dew point temperatures, max and mean wind speed, global irradiance and direct irradiance.  

#### Step 1a: Select location of interest
TMYs are calculated for a specific location of interest, like a building or power plant. Here, we will use a known weather station location, via their latitude and longitude to extract the data that we need to calculate the TMY. In the example below, we will look specifically at Los Angeles International Airport, but will note in the code below how you can provide your own location coordinates too. 

In [ ]:
stn_name = "Los Angeles International Airport (KLAX)"
# Show how to see all station names

#### Step 1b: Select time frame of interest
The second required input for generating a TMY dataset is the **time frame of interest**. The recommended minimum number of input years for a TMY dataset is 15-20 years worth of daily data; we will use 30 years to represent a standard climatological period. For data post-2014, we will utilize SSP 3-7.0, although scenario selection in the near-future is relatively independent. If calculating a TMY for the far-future, **carefully consider which scenario SSP to include**, as there will be **significant** differences present. 

We will also process the data for our designated station location (latitude, and longitude) at 9 km over the <span style="color:#FF0000">1990-2020 period</span> as an example. **Note**: An additional year (2021) is also loaded to pad the end of the dataset after converting to local time in the next few steps -- this is done for you when subsetting for the data. 

In [ ]:
# selected reference period
start_year = 1990
end_year = 2020

### Step 2: Generate the TMY data outputs

Generally, the following data is outputted using the TMY months:
- Date & time (UTC)
- Air temperature at 2m [°C]
- Dew point temperature [°C]
- Relative humidity [%]
- Global horizontal irradiance [W/m2]
- Direct normal irradiance [W/m2]
- Diffuse horizontal irradiance [W/m2]
- Downwelling infrared radiation [W/m2]
- Wind speed at 10m [m/s]
- Wind direction at 10m [°]
- Surface air pressure [Pa]

We can use the TMY object to set up, run, and output TMY results to file. The first step is to initialize the object with your desired reference period (`start_year` and `end_year`) and location (`stn_name`).  

In [ ]:
# Initialize TMY object
tmy = TMY(start_year, end_year, stn_name, verbose=True)

We can run the entire TMY workflow with a single command, as shown below. This will write 4 TMY files, one for each model.  
The runtime for this command can reach up to **30 minutes**. Because we set `verbose` to True, the class will print updates as different parts of the workflow initialize.

In [ ]:
tmy.generate_tmy()

Let's observe what the TMY data looks like for one of the simulations:

In [ ]:
# view tmy_data_to_export

Next, we visualize the TMY data itself. We show the results for a single simulation:

In [ ]:
tmy.show_tmy_data_to_export("WRF_MPI-ESM1-2-HR_r3i1p1f1")

### Explore the TMY process

#### Models
It is important to note that not all models in the Cal-Adapt: Analytics Engine have the solar variables critical for TMY file generation - in fact, only 4 do! The TMY class subsets our variables to ensure that the same 4 models are selected for consistency. The simulations are shown below:

In [ ]:
tmy.data_models

### CDF

Now that we've run the TMY workflow, we can investigate the Cumulative Distribution Function results that were used to select top months.

In the plot below, we'll display maximum air temperature to assess the climatological CDF pattern, but you can modify the variable here to one of your choosing to see the pattern too! Also select a different month by moving the slider bar to see the pattern throughout the year. 

In [ ]:
# Choose your desired variable
var = "Daily max air temperature"

# Make the plot
cdf_plot = plot_one_var_cdf(tmy.cdf_climatology, var)
display(cdf_plot)

Like the climatology CDF figure above, let's check out the individual months next. You can modify the variable, and month-year to display too. 

In [ ]:
# Choose your desired variable
var = "Daily max air temperature"

# Make the plot
cdf_plot_mon_yr = plot_one_var_cdf(tmy.cdf_monthly,var)
display(cdf_plot_mon_yr)